# Load Transactions

In [22]:
from pathlib import Path
from datetime import datetime

import pandas as pd


def str2float(x):
    if not isinstance(x, str):
        return float(x)
    return float(x.replace(',', ''))

path = Path('/Users/ab011sw/Documents/Personal/investments/coinmate.tsv')
path = Path('/Users/ab011sw/Documents/Personal/investments/bittrex.tsv')
path = Path('/Users/ab011sw/Documents/Personal/investments/binance.tsv')

transactions = pd.read_csv(path, sep='\t')
transactions['Date'] = transactions['Date']\
    .apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))\

transactions['Price'] = transactions['Price'].apply(str2float)
transactions['Fee'] = transactions['Fee'].apply(str2float)

In [23]:
transactions.head()

,Date,Code,Currency,Price,Quantity,Action,Fee
0,2017-12-19,BTC-IOTA,BTC,0.000319,39,buy,0.0
1,2017-12-29,BTC-TRX,BTC,0.000002,6050,buy,0.0


# Process Data

In [24]:
from convertor.yahoo_convertor import YahooConvertor

convertors = {
    'CZK': YahooConvertor(ticker='CZK=X', inverse=True),
    'BTC': YahooConvertor(ticker='BTC-USD'),
    'ETH': YahooConvertor(ticker='ETH-USD'),
}

result = pd.DataFrame([], columns=transactions.columns)

for _, row in transactions.iterrows():
    convertor = convertors[row['Currency']]
    new_row = row.copy()
    new_row['Price'] = convertor.convert(row['Price'], row['Date'])
    new_row['Fee'] = convertor.convert(row['Fee'], row['Date'])
    new_row['Currency'] = 'USD'
    result = result.append(new_row)

result

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Code,Currency,Price,Quantity,Action,Fee
0,2017-12-19,BTC-IOTA,USD,5.670767,39,buy,0.0
1,2017-12-29,BTC-TRX,USD,0.035468,6050,buy,0.0


In [25]:
out_path = path.parent / f'{path.stem}_usd.csv'

result.to_csv(out_path)